In [12]:
!nvidia-smi

Wed Mar 20 23:41:52 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.14              Driver Version: 550.54.14      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 2070 ...    Off |   00000000:0A:00.0  On |                  N/A |
|  0%   49C    P8             23W /  215W |     478MiB /   8192MiB |     23%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import opendatasets as od
import string
import random
import os

from collections import Counter

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import Normalize, ToTensor

In [2]:
if torch.cuda.is_available():
    device=torch.device(type="cuda", index=0)
else:
    device=torch.device(type="cpu", index=0)
print(device)

cuda:0


In [4]:
dataset_path = "Datasets/final.csv"
org_dataset = pd.read_csv(dataset_path)
org_dataset.head()

,Unnamed: 0,id,og,t
0,0,42928-1500614319216-63344,You do not meet a man but frowns:,Every man you meet these days is frowning.
1,1,42928-1500614326583-89821,our bloods No more obey the heavens than our...,Our bodies are in agreement with the planetar...
2,2,A-63849,But what's the matter?,What's wrong?
3,3,42930-1500614347266-80123,"His daughter, and the heir of's kingdom, whom...","The king wanted his daughter, the only heir to..."
4,4,42930-1500614355280-38326,she's wedded; Her husband banish'd; she impr...,"She's married, her husband is banished, she's..."


In [10]:

assoc_dict = dict(zip(org_dataset['t'], org_dataset['og']))
# print(assoc_dict.items())

In [ ]:
class ShakespeareDataset(Dataset):
    def __init__(self, file_path) -> None:
        super().__init__()
        self.data = self.loadData(file_path)
        self.vocab = self.getVocab()
        
    def loadData(self, file_path):
        # Preprocess as well
        with open(file_path, 'r', encoding='utf-8') as file:
            data = file.read()
        lines = data.split('\n')
        lines = [line.strip(r'\"') for line in lines]
        words = ' '.join(lines).split()
        punct = set(string.punctuation)
        words = [''.join([char for char in list(word) if char not in punct]) for word in words]
        return words
    
    def getVocab(self):
        char_counts = Counter(self.data)
        vocab = {char: idx for idx, (char, _) in enumerate(sorted(char_counts.items()))}
        vocab_size = len(vocab)
        return vocab, vocab_size
    
    def __len__(self):
        return len(self.data)
    
    # def __getitem__(self, index):
    #     return 

In [1]:
from models.models import Encoder, Decoder, EncDec

/usr/local/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
e = Encoder(3,2,2,'lstm')
print(e)

Encoder(
  (rnn): LSTM(3, 2, num_layers=2)
)
